# งานการจัดประเภทข้อความ

ในโมดูลนี้ เราจะเริ่มต้นด้วยงานการจัดประเภทข้อความง่าย ๆ โดยใช้ชุดข้อมูล **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: เราจะจัดประเภทหัวข้อข่าวให้อยู่ในหนึ่งใน 4 หมวดหมู่ ได้แก่ โลก กีฬา ธุรกิจ และวิทยาศาสตร์/เทคโนโลยี

## ชุดข้อมูล

เพื่อโหลดชุดข้อมูล เราจะใช้ API ของ **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

เราสามารถเข้าถึงส่วนการฝึกอบรมและการทดสอบของชุดข้อมูลได้โดยใช้ `dataset['train']` และ `dataset['test']` ตามลำดับ:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


มาพิมพ์พาดหัวข่าวใหม่ 10 หัวข้อแรกจากชุดข้อมูลของเรากัน:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## การแปลงข้อความเป็นเวกเตอร์

ตอนนี้เราต้องแปลงข้อความให้เป็น **ตัวเลข** ที่สามารถแสดงผลในรูปแบบเทนเซอร์ได้ หากเราต้องการการแสดงผลในระดับคำ เราต้องทำสองสิ่ง:

* ใช้ **tokenizer** เพื่อแบ่งข้อความออกเป็น **tokens**  
* สร้าง **vocabulary** ของ tokens เหล่านั้น  

### การจำกัดขนาดของคำศัพท์

ในตัวอย่างชุดข้อมูล AG News ขนาดของคำศัพท์ค่อนข้างใหญ่ มีมากกว่า 100,000 คำ โดยทั่วไปแล้ว เราไม่จำเป็นต้องใช้คำที่ปรากฏในข้อความน้อยมาก — มีเพียงไม่กี่ประโยคเท่านั้นที่จะมีคำเหล่านั้น และโมเดลจะไม่สามารถเรียนรู้จากคำเหล่านั้นได้ ดังนั้นจึงสมเหตุสมผลที่จะจำกัดขนาดของคำศัพท์ให้เล็กลงโดยการส่งอาร์กิวเมนต์ไปยังตัวสร้าง vectorizer:

ทั้งสองขั้นตอนนี้สามารถจัดการได้โดยใช้เลเยอร์ **TextVectorization** มาสร้างออบเจ็กต์ vectorizer และเรียกใช้เมธอด `adapt` เพื่อประมวลผลข้อความทั้งหมดและสร้างคำศัพท์:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **หมายเหตุ** เราใช้เพียงส่วนหนึ่งของชุดข้อมูลทั้งหมดในการสร้างคำศัพท์ เพื่อเพิ่มความเร็วในการประมวลผลและไม่ให้คุณต้องรอนาน อย่างไรก็ตาม เรากำลังรับความเสี่ยงที่คำบางคำจากชุดข้อมูลทั้งหมดอาจไม่ได้ถูกรวมอยู่ในคำศัพท์ และจะถูกละเว้นในระหว่างการฝึกอบรม ดังนั้น การใช้ขนาดคำศัพท์ทั้งหมดและประมวลผลผ่านชุดข้อมูลทั้งหมดในระหว่าง `adapt` อาจช่วยเพิ่มความแม่นยำสุดท้ายได้ แต่จะไม่เพิ่มขึ้นอย่างมีนัยสำคัญ

ตอนนี้เราสามารถเข้าถึงคำศัพท์จริงได้แล้ว:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


โดยใช้ตัวเวคเตอไรเซอร์ เราสามารถเข้ารหัสข้อความใด ๆ ให้เป็นชุดของตัวเลขได้อย่างง่ายดาย:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## การแสดงข้อความแบบ Bag-of-words

เนื่องจากคำมีความหมายในตัวเอง บางครั้งเราสามารถเข้าใจความหมายของข้อความได้เพียงแค่มองที่คำแต่ละคำ โดยไม่ต้องสนใจลำดับของคำในประโยค ตัวอย่างเช่น เมื่อเราจำแนกข่าว คำอย่าง *weather* และ *snow* มักจะบ่งบอกถึง *พยากรณ์อากาศ* ในขณะที่คำอย่าง *stocks* และ *dollar* จะเกี่ยวข้องกับ *ข่าวการเงิน* 

**Bag-of-words** (BoW) เป็นการแสดงข้อความในรูปแบบเวกเตอร์ที่เข้าใจได้ง่ายที่สุดในบรรดาการแสดงข้อความแบบเวกเตอร์แบบดั้งเดิม โดยแต่ละคำจะถูกเชื่อมโยงกับดัชนีในเวกเตอร์ และแต่ละองค์ประกอบในเวกเตอร์จะบอกจำนวนครั้งที่คำแต่ละคำปรากฏในเอกสารที่กำหนด

![ภาพแสดงการแสดงข้อความแบบ bag-of-words ในหน่วยความจำ](../../../../../translated_images/th/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: คุณสามารถคิดถึง BoW ว่าเป็นผลรวมของเวกเตอร์แบบ one-hot-encoded ของคำแต่ละคำในข้อความก็ได้เช่นกัน

ด้านล่างนี้คือตัวอย่างการสร้างการแสดงข้อความแบบ bag-of-words โดยใช้ไลบรารี Scikit Learn ในภาษา Python:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

เรายังสามารถใช้ตัวเวกเตอไรเซอร์ของ Keras ที่เราได้กำหนดไว้ข้างต้น โดยแปลงแต่ละหมายเลขคำให้เป็นการเข้ารหัสแบบหนึ่งร้อนและรวมเวกเตอร์เหล่านั้นทั้งหมดเข้าด้วยกัน:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **หมายเหตุ**: คุณอาจแปลกใจที่ผลลัพธ์แตกต่างจากตัวอย่างก่อนหน้า สาเหตุคือในตัวอย่างของ Keras ความยาวของเวกเตอร์สอดคล้องกับขนาดของคำศัพท์ ซึ่งถูกสร้างขึ้นจากชุดข้อมูล AG News ทั้งหมด ในขณะที่ในตัวอย่างของ Scikit Learn เราสร้างคำศัพท์จากข้อความตัวอย่างแบบทันที


## การฝึกโมเดล BoW classifier

ตอนนี้เราได้เรียนรู้วิธีสร้างตัวแทนแบบ bag-of-words สำหรับข้อความของเราแล้ว มาฝึกโมเดล classifier ที่ใช้ตัวแทนนี้กัน ก่อนอื่น เราจำเป็นต้องแปลงชุดข้อมูลของเราให้อยู่ในรูปแบบ bag-of-words ซึ่งสามารถทำได้โดยใช้ฟังก์ชัน `map` ในลักษณะดังนี้:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

ตอนนี้เรามากำหนดเครือข่ายประสาทตัวจำแนกแบบง่ายที่มีเพียงชั้นเชิงเส้นหนึ่งชั้น ขนาดของอินพุตคือ `vocab_size` และขนาดของเอาต์พุตสอดคล้องกับจำนวนคลาส (4) เนื่องจากเรากำลังแก้ปัญหาการจำแนกประเภท ฟังก์ชันการกระตุ้นสุดท้ายคือ **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


เนื่องจากเรามี 4 คลาส ความแม่นยำที่มากกว่า 80% ถือว่าเป็นผลลัพธ์ที่ดี

## การฝึกโมเดลจำแนกประเภทเป็นเครือข่ายเดียว

เนื่องจากตัวแปลงเวกเตอร์ก็เป็นเลเยอร์ของ Keras เช่นกัน เราสามารถกำหนดเครือข่ายที่รวมตัวแปลงเวกเตอร์ไว้ และฝึกมันแบบ end-to-end ด้วยวิธีนี้เราไม่จำเป็นต้องแปลงข้อมูลชุดด้วย `map` เราสามารถส่งข้อมูลชุดต้นฉบับไปยังอินพุตของเครือข่ายได้เลย

> **Note**: เรายังคงต้องใช้ `map` กับข้อมูลชุดของเราเพื่อแปลงฟิลด์จากดิกชันนารี (เช่น `title`, `description` และ `label`) เป็นทูเพิล อย่างไรก็ตาม เมื่อโหลดข้อมูลจากดิสก์ เราสามารถสร้างข้อมูลชุดที่มีโครงสร้างที่ต้องการตั้งแต่แรกได้


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## ไบแกรม, ไตรแกรม และเอ็นแกรม

ข้อจำกัดอย่างหนึ่งของวิธีการแบบถุงคำ (bag-of-words) คือบางคำเป็นส่วนหนึ่งของวลีที่มีหลายคำ ตัวอย่างเช่น คำว่า 'hot dog' มีความหมายที่แตกต่างอย่างสิ้นเชิงจากคำว่า 'hot' และ 'dog' ในบริบทอื่นๆ หากเราแทนคำว่า 'hot' และ 'dog' ด้วยเวกเตอร์เดียวกันเสมอ อาจทำให้โมเดลของเราสับสนได้

เพื่อแก้ปัญหานี้ **การแทนค่าด้วยเอ็นแกรม** มักถูกนำมาใช้ในวิธีการจัดประเภทเอกสาร โดยที่ความถี่ของแต่ละคำ, คำคู่ หรือคำสามคำ เป็นคุณลักษณะที่มีประโยชน์สำหรับการฝึกตัวจำแนก ในการแทนค่าด้วยไบแกรม ตัวอย่างเช่น เราจะเพิ่มคู่คำทั้งหมดลงในคลังคำ นอกเหนือจากคำเดิมที่มีอยู่แล้ว

ด้านล่างนี้คือตัวอย่างวิธีการสร้างการแทนค่าถุงคำแบบไบแกรมโดยใช้ Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

ข้อเสียหลักของวิธี n-gram คือขนาดของคำศัพท์ที่เพิ่มขึ้นอย่างรวดเร็วมาก ในทางปฏิบัติ เราจำเป็นต้องรวมการแสดงผลแบบ n-gram เข้ากับเทคนิคการลดมิติ เช่น *embeddings* ซึ่งเราจะพูดถึงในบทถัดไป

ในการใช้การแสดงผลแบบ n-gram กับชุดข้อมูล **AG News** เราจำเป็นต้องส่งพารามิเตอร์ `ngrams` ไปยังตัวสร้าง `TextVectorization` ขนาดของคำศัพท์แบบ bigram นั้น **ใหญ่ขึ้นอย่างมาก** ในกรณีของเรา มีมากกว่า 1.3 ล้านคำ! ดังนั้นจึงสมเหตุสมผลที่จะจำกัดคำแบบ bigram ด้วยจำนวนที่เหมาะสม

เราสามารถใช้โค้ดเดียวกับด้านบนเพื่อฝึกตัวจำแนกประเภทได้ อย่างไรก็ตาม วิธีนี้จะใช้หน่วยความจำอย่างไม่มีประสิทธิภาพ ในบทถัดไป เราจะฝึกตัวจำแนกประเภทแบบ bigram โดยใช้ embeddings ในระหว่างนี้ คุณสามารถทดลองฝึกตัวจำแนกประเภทแบบ bigram ในโน้ตบุ๊กนี้และดูว่าคุณสามารถเพิ่มความแม่นยำได้หรือไม่


## การคำนวณเวกเตอร์ BoW โดยอัตโนมัติ

ในตัวอย่างข้างต้น เราได้คำนวณเวกเตอร์ BoW ด้วยมือโดยการรวมผลรวมของการเข้ารหัสแบบ one-hot ของแต่ละคำ อย่างไรก็ตาม เวอร์ชันล่าสุดของ TensorFlow ช่วยให้เราสามารถคำนวณเวกเตอร์ BoW ได้โดยอัตโนมัติ เพียงแค่เพิ่มพารามิเตอร์ `output_mode='count` ลงในตัวสร้าง vectorizer วิธีนี้ทำให้การกำหนดและฝึกโมเดลของเราง่ายขึ้นอย่างมาก:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## ความถี่คำ - ความถี่เอกสารผกผัน (TF-IDF)

ในรูปแบบการแสดงผลแบบ BoW การนับจำนวนคำจะถูกถ่วงน้ำหนักด้วยเทคนิคเดียวกันโดยไม่คำนึงถึงตัวคำเอง อย่างไรก็ตาม เป็นที่ชัดเจนว่าคำที่พบได้บ่อย เช่น *a* และ *in* มีความสำคัญน้อยกว่าคำเฉพาะทางสำหรับการจัดประเภท ในงาน NLP ส่วนใหญ่ บางคำมีความเกี่ยวข้องมากกว่าคำอื่นๆ

**TF-IDF** ย่อมาจาก **term frequency - inverse document frequency** ซึ่งเป็นรูปแบบหนึ่งของ bag-of-words โดยแทนที่จะใช้ค่าไบนารี 0/1 เพื่อบ่งบอกการปรากฏของคำในเอกสาร จะใช้ค่าทศนิยมที่สัมพันธ์กับความถี่ของการปรากฏของคำในคอร์ปัสแทน

ในเชิงคณิตศาสตร์ น้ำหนัก $w_{ij}$ ของคำ $i$ ในเอกสาร $j$ ถูกกำหนดเป็น:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
โดยที่
* $tf_{ij}$ คือจำนวนครั้งที่คำ $i$ ปรากฏในเอกสาร $j$ หรือค่าของ BoW ที่เราเคยเห็นมาก่อน
* $N$ คือจำนวนเอกสารทั้งหมดในชุดข้อมูล
* $df_i$ คือจำนวนเอกสารที่มีคำ $i$ ในชุดข้อมูลทั้งหมด

ค่าของ TF-IDF $w_{ij}$ จะเพิ่มขึ้นตามสัดส่วนของจำนวนครั้งที่คำปรากฏในเอกสาร และจะถูกปรับลดลงตามจำนวนเอกสารในคอร์ปัสที่มีคำดังกล่าว ซึ่งช่วยปรับให้เหมาะสมกับความจริงที่ว่าบางคำปรากฏบ่อยกว่าคำอื่นๆ ตัวอย่างเช่น หากคำปรากฏใน *ทุก* เอกสารในชุดข้อมูล $df_i=N$ และ $w_{ij}=0$ คำเหล่านั้นจะถูกละเลยไปโดยสิ้นเชิง

คุณสามารถสร้างการเวกเตอร์ TF-IDF ของข้อความได้อย่างง่ายดายโดยใช้ Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

ใน Keras ชั้น `TextVectorization` สามารถคำนวณความถี่ TF-IDF ได้โดยอัตโนมัติโดยการส่งพารามิเตอร์ `output_mode='tf-idf'` ลองทำซ้ำโค้ดที่เราใช้ด้านบนเพื่อดูว่าการใช้ TF-IDF เพิ่มความแม่นยำหรือไม่:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## บทสรุป

แม้ว่า TF-IDF จะช่วยให้ค่าน้ำหนักความถี่กับคำต่าง ๆ ได้ แต่ก็ยังไม่สามารถแสดงถึงความหมายหรือเรียงลำดับของคำได้ ดังที่นักภาษาศาสตร์ชื่อดัง J. R. Firth กล่าวไว้ในปี 1935 ว่า "ความหมายที่สมบูรณ์ของคำใด ๆ นั้นขึ้นอยู่กับบริบทเสมอ และการศึกษาความหมายที่แยกออกจากบริบทไม่สามารถถือว่าเป็นเรื่องจริงจังได้" เราจะได้เรียนรู้วิธีการจับข้อมูลเชิงบริบทจากข้อความโดยใช้การสร้างแบบจำลองภาษาในภายหลังในคอร์สนี้



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
